In [1]:
from selenium import webdriver # 1 login 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import pyautogui
import pandas as pd
import openpyxl
from PyPDF2 import PdfReader, PdfWriter
import fitz
import os, os.path
import pathlib
from pathlib import Path
import shutil

#get user input
import tkinter as tk

def get_username():
    root = tk.Tk()
    root.title("Esker Invoice Download")
    root.geometry("350x200")

    name_var = tk.StringVar()
    username=None

    def submit_username():
        nonlocal username
        username = name_var.get()
        print("Username:", username)  # Print username within the function
        root.destroy()  # Close the window after submission
        #return name  # Return the username

    name_label = tk.Label(root, text='Username', font=('calibre', 10, 'bold'))
    name_entry = tk.Entry(root, textvariable=name_var, font=('calibre', 10, 'normal'))

    submit_btn = tk.Button(root, text='Submit', command=submit_username)

    name_label.grid(row=0, column=0, padx=5, pady=5)
    name_entry.grid(row=0, column=1, padx=5, pady=5)
    submit_btn.grid(row=1, column=1, padx=5, pady=5)
    root.mainloop()  # Start the Tkinter event loop and store the returned value
    return username  # Return the username

global username
username=get_username()  # Call the function to get the username


driver = webdriver.Chrome()
driver.get("https://az3.ondemand.esker.com/ondemand/webaccess/asf/home.aspx")
driver.maximize_window()
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="ctl03_tbUser"]').send_keys("john.tan@sh-cogent.com.sg")
driver.find_element(By.XPATH, '//*[@id="ctl03_tbPassword"]').send_keys("Esker3838")
driver.find_element(By.XPATH, '//*[@id="ctl03_btnSubmitLogin"]').click()
time.sleep(5) # login esker



Username: john.tan


In [ ]:
no_items_to_display = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td')
no_items_to_display.get_attribute('innerHTML')


'There are no items to display'

In [5]:
list_invoice_numbr = ['1525007348'] #'305498'
list_invoice_numbr_on_page = []

def items_to_display():
    i = 0 #line 1
    idx_invoice_numbr = str(i+2)
    try:
            invoice_numbr_on_page = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr['+idx_invoice_numbr+']/td[7]/a')
            print(f"{invoice_numbr} found: "+invoice_numbr_on_page.get_attribute("innerHTML"))
            list_invoice_numbr_on_page.append(str(invoice_numbr_on_page.get_attribute("innerHTML")))
              
    except Exception as e:
            no_items_to_display = driver.find_element(By.XPATH, '//*[@id="tpl_ih_adminList_MainGrid"]/tbody/tr[2]/td')
            print(f"{invoice_numbr} not found: "+no_items_to_display.get_attribute('innerHTML'))
    print(f"list_invoice_numbr_on_page: {list_invoice_numbr_on_page}")
    return list_invoice_numbr_on_page    
          
for invoice_numbr in list_invoice_numbr:
            #reset_input_invoice_numbr(invoice_numbr)
            time.sleep(1.0)
            items_to_display()
            continue

1525007348 found: 1525007348
list_invoice_numbr_on_page: ['1525007348']


In [ ]:
list_invoice_numbr_on_page = ['1525007348'] #'305498'
dict_invoice_numbr = {'1525007348': 0, '305498': 0}

def filter_dict_by_list(list_invoice_numbr_on_page, dict_invoice_numbr):
    """
    Create a new dictionary containing only keys that exist in the given list.
    Args:
        list_invoice_numbr_on_page (list): List of invoice numbers to filter by
        dict_invoice_numbr (dict): Original dictionary to filter
    Returns:
        dict: New dictionary containing only keys from the list
    """
    dict_invoice_numbr_on_page = {
        key: dict_invoice_numbr[key] 
        for key in list_invoice_numbr_on_page 
        if key in dict_invoice_numbr
    }
    return dict_invoice_numbr_on_page

# Example usage:
list_invoice_numbr_on_page = ['1525007348']
dict_invoice_numbr = {'1525007348': 0, '305498': 0}
# Call the function
result = filter_dict_by_list(list_invoice_numbr_on_page, dict_invoice_numbr)
print(result)  # Output: {'1525007348': 0}



{'1525007348': 0}


In [1]:
from diagrams import Cluster, Diagram
from diagrams.aws.compute import ECS, EKS, Lambda
from diagrams.aws.database import Redshift
from diagrams.aws.integration import SQS
from diagrams.aws.storage import S3

with Diagram("Event Processing", show=False):
    source = EKS("k8s source")

    with Cluster("Event Flows"):
        with Cluster("Event Workers"):
            workers = [ECS("worker1"),
                       ECS("worker2"),
                       ECS("worker3")]

        queue = SQS("event queue")

        with Cluster("Processing"):
            handlers = [Lambda("proc1"),
                        Lambda("proc2"),
                        Lambda("proc3")]

    store = S3("events store")
    dw = Redshift("analytics")

    source >> workers >> queue >> handlers
    handlers >> store
    handlers >> dw
    

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH